In [12]:
# Base code for only optimization purposes

from gurobipy import Model
from gurobipy import GRB

# Load data from the provided JSON file
import json
with open('MWM.json', 'r') as f:
    data = json.load(f)

# Sets and parameters
recipients = list(set(entry["Recipient"] for entry in data))
donors = list({d for entry in data for d in entry["Donor"]})
compatibility = {(entry["Recipient"], d): 1 for entry in data for d in entry["Donor"]}

# Create model
model = Model("Kidney_Paired_Donation")

# Decision variables
x = model.addVars(compatibility.keys(), vtype=GRB.BINARY, name="x")

# Objective function
model.setObjective(x.sum(), GRB.MAXIMIZE)

# Constraints
for i in recipients:
    model.addConstr(sum(x[i, j] for j in donors if (i, j) in compatibility) <= 1)

for j in donors:
    model.addConstr(sum(x[i, j] for i in recipients if (i, j) in compatibility) <= 1)

# Optimize the model
model.optimize()

# Results
if model.status == GRB.OPTIMAL:
    for i, j in compatibility:
        if x[i, j].x > 0.5:
            print(f"Recipient {i} is matched with Donor {j}")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 6 rows, 7 columns and 14 nonzeros
Model fingerprint: 0x96df0e66
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 6 rows and 7 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 0.0000%
Recipient A is matched with Donor B
Recipient 

In [25]:
# Code to find cycles of size 2 and 3

from gurobipy import Model
from gurobipy import GRB
from gurobipy import quicksum
import json

# Load the dataset
with open('MWM.json', 'r') as f:
    data = json.load(f)

# Prepare sets and parameters
recipients = list(set(entry["Recipient"] for entry in data))
donors = list({d for entry in data for d in entry["Donor"]})
compatibility = {(entry["Recipient"], d): 1 for entry in data for d in entry["Donor"]}

# Creating a graph to help show compatibility
edges = [(r, d) for r, d in compatibility.keys()]

# Create the model
model = Model("Cycles_Size_2_and_3")

# Decision variables: x[i, j] shows if donor j donates to recipient i
x = model.addVars(edges, vtype=GRB.BINARY, name="x")

# Objective: Maximize the total number of transplants
model.setObjective(quicksum(x[i, j] for i, j in edges), GRB.MAXIMIZE)

# Constraint 1: Each recipient gets at most one kidney
for i in recipients:
    model.addConstr(quicksum(x[i, j] for j in donors if (i, j) in edges) <= 1)

# Constraint 2: Each donor donates at most one kidney
for j in donors:
    model.addConstr(quicksum(x[i, j] for i in recipients if (i, j) in edges) <= 1)

# Additional constraints to enforce cycles of size 2 or 3
for i in recipients:
    for j in donors:
        if (i, j) in edges:  # Ensure (i, j) exists in edges
            for k in recipients:
                if k != i and (j, k) in edges:
                    # Enforce cycle size 2
                    model.addConstr(x[i, j] + x[j, k] <= 1)  # No overlapping edges
                for l in donors:
                    if l != j and (k, l) in edges and (l, i) in edges:
                        # Enforce cycle size 3 only if all edges exist
                        if (i, j) in edges and (j, k) in edges and (k, l) in edges and (l, i) in edges:
                            model.addConstr(
                                x[i, j] + x[j, k] + x[k, l] + x[l, i] <= 1
                            )

# Solving the model
model.optimize()

# Displaying the results
if model.status == GRB.OPTIMAL:
    print("\nOptimal cycles of size 2 and 3:")
    for i, j in edges:
        if x[i, j].x > 0.5:
            print(f"Recipient {i} <- Donor {j}")
else:
    print("No optimal solution found.")

Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: Intel(R) Core(TM) i7-7700HQ CPU @ 2.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 8 rows, 7 columns and 18 nonzeros
Model fingerprint: 0x3eae04db
Variable types: 0 continuous, 7 integer (7 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 13 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 8 (of 8 available processors)

Solution count 1: 3 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e+00, best bound 3.000000000000e+00, gap 